# Projeto 06 - Safra 😎

| Coluna            | Tipo da variável      |                                           Descrição | Utilizada |
| :---------------- | :-------------------- | --------------------------------------------------: | :-------: |
| LoanNr_ChkDgt     | Quantitativa Discreta |                              Identifier Primary key |    ❌     |
| Name              | Qualitativa Nominal   |                                       Borrower name |    ✅     |
| City              | Qualitativa Nominal   |                                       Borrower city |    ✅     |
| State             | Qualitativa Nominal   |                                      Borrower state |    ✅     |
| Zip               | Qualitativa Nominal   |                                   Borrower zip code |    ❌     |
| Bank              | Qualitativa Nominal   |                                           Bank name |    ✅     |
| BankState         | Qualitativa Nominal   |                                          Bank state |    ❌     |
| NAICS             | Qualitativa Ordinal   |  North American industry classification system code |    ✅     |
| ApprovalDate      | Qualitativa Ordinal   |                          Date SBA commitment issued |    ✅     |
| ApprovalFY        | Quantitativa Discreta |                           Fiscal year of commitment |    ✅     |
| Term              | Quantitativa Discreta |                                 Loan term in months |    ✅     |
| NoEmp             | Quantitativa Discreta |                        Number of business employees |    ✅     |
| NewExist          | Qualitativa Nominal   |             1 = Existing business, 2 = New business |    ✅     |
| CreateJob         | Quantitativa Discreta |                              Number of jobs created |    ❌     |
| RetainedJob       | Quantitativa Discreta |                             Number of jobs retained |    ❌     |
| FranchiseCode     | Qualitativa Nominal   |     Franchise code, (00000 or 00001) = No franchise |    ❌     |
| UrbanRural        | Qualitativa Nominal   |                 1 = Urban, 2 = rural, 0 = undefined |    ✅     |
| RevLineCr         | Qualitativa Nominal   |           Revolving line of credit: Y = Yes, N = No |    ✅     |
| LowDoc            | Qualitativa Nominal   |                LowDoc Loan Program: Y = Yes, N = No |    ✅     |
| ChgOffDate        | Qualitativa Ordinal   |   The date when a loan is declared to be in default |    ✅     |
| DisbursementDate  | Qualitativa Ordinal   |                                   Disbursement date |    ✅     |
| DisbursementGross | Quantitativa Contínua |                                    Amount disbursed |    ✅     |
| BalanceGross      | Quantitativa Contínua |                            Gross amount outstanding |    ✅     |
| MIS_Status        | Qualitativa Nominal   | Loan status charged off = CHGOFF, Paid in full =PIF |    ✅     |
| ChgOffPrinGr      | Quantitativa Contínua |                                  Charged-off amount |    ✅     |
| GrAppv            | Quantitativa Contínua |               Gross amount of loan approved by bank |    ✅     |
| SBA_Appv          | Quantitativa Contínua |            SBA’s guaranteed amount of approved loan |    ✅     |



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_columns', None)

In [ ]:
df_raw = pd.read_csv("SBAnational.csv")
df_raw.head()

In [ ]:
df_raw.info()

## Tratamento dos dados

In [ ]:
df_raw.isna().sum()

In [ ]:
df_raw.dropna(subset=["Name", "City", "State", "Bank", "BankState", "NewExist", "RevLineCr", "LowDoc", "DisbursementDate", "MIS_Status"], inplace=True)

In [ ]:
df_raw.drop(columns=["CreateJob", "RetainedJob", "FranchiseCode", "Zip", "BankState", "LoanNr_ChkDgt"], inplace=True)

In [ ]:
df_raw.isna().sum()

In [ ]:
money_type_columns = ["DisbursementGross", "BalanceGross", "ChgOffPrinGr", "GrAppv", "SBA_Appv"]
for col in money_type_columns:
    df_raw[col] = df_raw[col].apply(lambda x: x.replace("$", "").replace(",", "")).astype(float)

In [ ]:
df_raw["ApprovalDate"] = pd.to_datetime(df_raw["ApprovalDate"])
df_raw["DisbursementDate"] = pd.to_datetime(df_raw["DisbursementDate"])

In [ ]:
df_raw = df_raw[df_raw["LowDoc"].isin(("Y", "N"))]
df_raw = df_raw[df_raw["RevLineCr"].isin(("Y", "N"))]

In [ ]:
df_raw["ApprovalFY"].replace("1976A", 1976, inplace=True)
df_raw["ApprovalFY"] = df_raw["ApprovalFY"].astype(int)

In [ ]:
df_raw["NewExist"] = df_raw["NewExist"].astype(int)

## Variáveis Qualitativas // Categorical

| Coluna            | Tipo da variável                   |                                           Descrição |
| :---------------- | :--------------------------------- | --------------------------------------------------: |
| Name              | Qualitativa Nominal                |                                       Borrower name |
| City              | Qualitativa Nominal                |                                       Borrower city |
| State             | Qualitativa Nominal                |                                      Borrower state |
| Bank              | Qualitativa Nominal                |                                           Bank name |
| NAICS             | Qualitativa Ordinal                |  North American industry classification system code |
| ApprovalDate      | Qualitativa Ordinal                |                          Date SBA commitment issued |
| NewExist          | Qualitativa Nominal                |             1 = Existing business, 2 = New business |
| UrbanRural        | Qualitativa Nominal                |                 1 = Urban, 2 = rural, 0 = undefined |
| RevLineCr         | Qualitativa Nominal                |           Revolving line of credit: Y = Yes, N = No |
| LowDoc            | Qualitativa Nominal                |                LowDoc Loan Program: Y = Yes, N = No |
| ChgOffDate        | Qualitativa Ordinal                |   The date when a loan is declared to be in default |
| MIS_Status        | Qualitativa Nominal                | Loan status charged off = CHGOFF, Paid in full =PIF |
| DisbursementDate  | Qualitativa Ordinal                |                                   Disbursement date |

In [ ]:
# Groupby values more frequent grouped by 'city'
city_group = df_raw.groupby("City").size().sort_values(ascending=False)

# Plot an bar chart
city_group.head(10).plot(kind="bar", figsize=(10, 5), title="Top 10 cities with more loans")

In [ ]:
# Plot a mean of loan amount by city
city_group = df_raw.groupby("City")["DisbursementGross"].mean().sort_values(ascending=False)
city_group.head(10).plot(kind="bar", figsize=(10, 5), title="")

In [ ]:
categorical_columns = ["State", "RevLineCr", "LowDoc", "NewExist", "MIS_Status"]

fig, axs = plt.subplots(3, 2, figsize=(15, 10))

for col, ax in zip(categorical_columns, axs.flatten()):
    df_raw[col].value_counts().plot(kind="bar", title=col, ax=ax)
    plt.xticks(rotation = 45)
plt.tight_layout()


## Variáveis Quantitativas

| Coluna            | Tipo da variável                   |                                           Descrição |
| :---------------- | :--------------------------------- | --------------------------------------------------: |
| GrAppv            | Quantitativa Contínua              |               Gross amount of loan approved by bank |
| DisbursementGross | Quantitativa Contínua              |                                    Amount disbursed |
| BalanceGross      | Quantitativa Contínua              |                            Gross amount outstanding |
| ApprovalFY        | Quantitativa Discreta              |                           Fiscal year of commitment |
| Term              | Quantitativa Discreta              |                                 Loan term in months |
| NoEmp             | Quantitativa Discreta              |                        Number of business employees |
| ChgOffPrinGr      | Quantitativa Contínua              |                                  Charged-off amount |
| SBA_Appv          | Quantitativa Contínua              |            SBA’s guaranteed amount of approved loan |




##  Iremos analisar de uma maneira completa as variáveis GrAppv, DisbursementGross, Term, NoEmp

| Coluna            | Tipo da variável                   |                                           Descrição |
| :---------------- | :--------------------------------- | --------------------------------------------------: |
| GrAppv            | Quantitativa Contínua              |               Gross amount of loan approved by bank |
| DisbursementGross | Quantitativa Contínua              |                                    Amount disbursed |
| Term              | Quantitativa Discreta              |                                 Loan term in months |
| NoEmp             | Quantitativa Discreta              |                        Number of business employees |


In [ ]:
# Create a function that appends to the new dataset all the items in describe() for each column in df_raw 
def describe_columns(df, colunas):
    df_describe = pd.DataFrame()
    for col in df.columns:
        if col in colunas:
            df_describe[col] = df[col].describe()
    return df_describe

cols = ["DisbursementGross", "GrAppv", "Term", "NoEmp"]
df_medidas_de_resumo = describe_columns(df_raw, cols)
display(df_medidas_de_resumo[cols])

### GrAppv
- [X] Medidas de Resumo
- [X] Maiores frequências e distribuição
- [X] Outliers e dizer a decisão
- [X] Gráfico


In [ ]:
# Medidas de Resumo

print(f'Medidas de resumo\n{df_raw["GrAppv"].describe()}')
print(f'\nMaiores frequências\n{df_raw["GrAppv"].value_counts().head(5)}')

In [ ]:
# Boxplot da coluna GrAppv
GrAppv = df_raw["GrAppv"]
boxplot = GrAppv.plot(kind="box", figsize=(10, 5))

In [ ]:
# Notamos que há vários outliers e criamos uma função que dividirá os valores em 2 grupos; o primeiro até a chamada linha de corte
#  e o segundo após a linha de corte (outliers superiores)

def interquartile(column):
    q1 = column.quantile(0.25)
    q3 = column.quantile(0.75) 
    iqr = q3 - q1
    return (q3 + 1.5 * iqr)

In [ ]:
linha_de_corte = interquartile(GrAppv)
linha_de_corte

In [ ]:
Dividas_ate_linhadecorte = GrAppv[GrAppv < linha_de_corte]
Dividas_ate_linhadecorte.describe()

In [ ]:
boxplot = Dividas_ate_linhadecorte.plot(kind="box", figsize=(10, 5))

In [ ]:
Dividas_apos_linhadecorte = GrAppv[GrAppv > linha_de_corte]
Dividas_apos_linhadecorte.describe()

In [ ]:
boxplot = Dividas_apos_linhadecorte.plot(kind="box", figsize=(10, 5))

In [ ]:
## Plot two histograms, one with linha_de_corte and other without
plot_1 = GrAppv[GrAppv < linha_de_corte].hist(bins=15, figsize=(10, 5), label="Sem linha_de_corte")
plt.ylabel("Frequência")
plt.xlabel("Crédito concedido")
plt.title(f'Valores até ${linha_de_corte:.0f}')

In [ ]:
# histogram the GrAppv column filtering by GrAppv 
GrAppvPlot = GrAppv[GrAppv > linha_de_corte].hist(bins=15, figsize=(10, 5))
plt.ylabel("Frequência")
plt. xlabel("Crédito concedido")
plt.title(f'Valores maiores que ${linha_de_corte:.0f}')

### DisbursementGross
- [X] Medidas de Resumo
- [X] Maiores frequências e distribuição
- [X] Outliers e dizer a decisão
- [X] Gráfico

In [ ]:
# Medidas de Resumo

print(f'Medidas de resumo\n{df_raw["DisbursementGross"].describe()}')
print(f'\nMaiores frequências\n{df_raw["DisbursementGross"].value_counts().head(5)}')

In [ ]:
# Boxplot da coluna DisbursementGross
DisbursementGross = df_raw["DisbursementGross"]
boxplot = DisbursementGross.plot(kind="box", figsize=(10, 5))

In [ ]:
# Notamos que há vários outliers e criamos uma função que dividirá os valores em 2 grupos; o primeiro até a chamada linha de corte
#  e o segundo após a linha de corte (outliers superiores)

def interquartile(column):
    q1 = column.quantile(0.25)
    q3 = column.quantile(0.75) 
    iqr = q3 - q1
    return (q3 + 1.5 * iqr)

In [ ]:
linha_de_corte = interquartile(DisbursementGross)
linha_de_corte

In [ ]:
Dividas_ate_linhadecorte = DisbursementGross[DisbursementGross < linha_de_corte]
Dividas_ate_linhadecorte.describe()

In [ ]:
boxplot = Dividas_ate_linhadecorte.plot(kind="box", figsize=(10, 5))

In [ ]:
Dividas_apos_linhadecorte = DisbursementGross[DisbursementGross > linha_de_corte]
Dividas_apos_linhadecorte.describe()

In [ ]:
boxplot = Dividas_apos_linhadecorte.plot(kind="box", figsize=(10, 5))

In [ ]:
## Plot two histograms, one with linha_de_corte and other without
plot_1 = DisbursementGross[DisbursementGross < linha_de_corte].hist(bins=15, figsize=(10, 5), label="Sem linha_de_corte")
plt.ylabel("Frequência")
plt.xlabel("Crédito concedido")
plt.title(f'Valores até ${linha_de_corte:.0f}')

In [ ]:
# histogram the DisbursementGross column filtering by DisbursementGross 
DisbursementGrossPlot = DisbursementGross[DisbursementGross > linha_de_corte].hist(bins=15, figsize=(10, 5))
plt.ylabel("Frequência")
plt.xlabel("Crédito concedido")
plt.title(f'Valores maiores que ${linha_de_corte:.0f}')

### Term
- [X] Medidas de Resumo
- [X] Maiores frequências e distribuição
- [X] Outliers e dizer a decisão
- [X] Gráfico

In [ ]:
# Medidas de Resumo

print(f'Medidas de resumo\n{df_raw["Term"].describe()}')
print(f'\nMaiores frequências\n{df_raw["Term"].value_counts().head(5)}')

In [ ]:
# Boxplot da coluna Term, optamos por não remover pois há empréstimos com um número maior de meses, o que não deve ser descartado.
Term = df_raw["Term"]
boxplot = Term.plot(kind="box", figsize=(10, 5))

In [ ]:
TermPlot = Term.hist(bins=15, figsize=(10, 5))
plt.ylabel("Quantidade de Empréstimos")
plt.xlabel("Quantidade de meses para pagamento")


### NoEmp
- [X] Medidas de Resumo
- [X] Maiores frequências e distribuição
- [X] Outliers e dizer a decisão
- [X] Gráfico

In [ ]:
# Medidas de Resumo

print(f'Medidas de resumo\n{df_raw["NoEmp"].describe()}')
print(f'\nMaiores frequências\n{df_raw["NoEmp"].value_counts().head(10)}')

In [ ]:
# Boxplot da coluna NoEmp
NoEmp = df_raw["NoEmp"]
boxplot = NoEmp.plot(kind="box", figsize=(10, 5))

In [ ]:
# Optamos por não retirar os outliers pois realmente existem empresas com grande número de funcionários
#  (sendo o maior valor 9999 - coerente para uma empresa de médio/grande porte), aplicamos a escala logaritmica no eixo Y para melhor visualização.
NoEmpPlot = NoEmp.hist(bins=100, figsize=(10, 5))
plt.ylabel("Quantidade de Empréstimos")
plt.xlabel("Quantidade de meses para pagamento")
plt.yscale('log')

In [ ]:
df_corr = df_raw[['GrAppv', 'DisbursementGross', 'Term', 'NoEmp']].corr()
df_corr

In [ ]:
sns.heatmap(df_corr, vmin=-1, vmax=1, cmap='coolwarm', center=None, robust=False, annot=True, fmt='.2g', annot_kws=None, linewidths=0.1, linecolor='white') 

## Parte 2

In [ ]:
valores = np.load('values_squad4.npy')
plt.hist(valores,bins=50, density=True); 

### Não aparenta ser normal mas faremos o teste

In [ ]:
# Cálculo da média e desvio padrão da AMOSTRA
media = np.mean(valores)
S = np.std(valores)

### TESTE DA DISTRIBUIÇÃO NORMAL

In [ ]:
#Criando uma normal com a média e desvio da amostra
normal = stats.norm(loc = media, scale=S)


In [ ]:
plt.hist(normal.rvs(size=1000),bins=40);

In [ ]:
k = 150
interval = np.linspace(np.min(valores), np.max(valores), k)
hist, bins = np.histogram(valores, bins=interval)

In [ ]:
# valores acumulados
normal_acumulada = normal.cdf(interval)
plt.plot(normal_acumulada)

In [ ]:
area_bins = normal_acumulada[1:] - normal_acumulada[:-1]
frequencia_teorica = area_bins * len(valores)

In [ ]:
sum(area_bins)

In [ ]:
plt.plot(frequencia_teorica)

In [ ]:
chi_amostra = np.sum((frequencia_teorica - hist)**2/(frequencia_teorica+1))
chi_amostra

In [ ]:
chi_critico = stats.chi2.ppf(0.95,df=k-2-1)
chi_critico

In [ ]:
if chi_amostra > chi_critico:
  print(f'Hipótese nula rejeitada pois chi_amostra ({chi_amostra}) > chi_critico ({chi_critico})')
else:
  print(f'Hipótese nula não rejeitada pois chi_amostra ({chi_amostra}) < chi_critico ({chi_critico})')